In [1]:
import numpy as np
import pandas as pd
import asyncio
import struct
import nest_asyncio
import bleak
from bleak import BleakScanner, BleakClient
from bleak.exc import BleakError
nest_asyncio.apply()
import re
from tqdm import tqdm
from queue import Queue

In [2]:
class DataCollector:
    def __init__(self, start_serial, end_serial):
        self.start_serial = start_serial
        self.end_serial = end_serial
        self.device_names = ["TD_" + str(i) for i in range(start_serial, end_serial+1)]
        self.start_len = len(self.device_names)
        self.df = pd.DataFrame({'Name': self.device_names, 
                                'MAC': [None] * len(self.device_names),
                                'RSSI': [None] * len(self.device_names),
                                'Oil Level' : [None] * len(self.device_names),
                                'Temperature': [None] * len(self.device_names),
                                'Battery Voltage': [None] * len(self.device_names),
                                'HK': [None] * len(self.device_names),
                                'LK': [None] * len(self.device_names)})
        
    def update_MAC(self, name, address):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'MAC'] is None:   
        self.df.at[index, 'MAC'] = address
            
    def update_RSSI(self, name, RSSI):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'RSSI'] is None:   
        self.df.at[index, 'RSSI'] = RSSI
            
    def update_oil_level(self, name, oil_level_raw):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'Oil Level'] is None:   
        self.df.at[index, 'Oil Level'] = oil_level_raw
            
    def update_temp(self, name, TD_temp_raw):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'Temperature'] is None:   
        self.df.at[index, 'Temperature'] = TD_temp_raw
            
    def update_battery_voltage(self, name, battery_voltage):
        index = self.df.index[self.df['Name'] == name][0]
        # if self.df.at[index, 'Battery Voltage'] is None:   
        self.df.at[index, 'Battery Voltage'] = battery_voltage
            
    def update_HK(self, name, hk):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'HK'] is None:   
        self.df.at[index, 'HK'] = hk
            
    def update_LK(self, name, lk):
        index = self.df.index[self.df['Name'] == name][0]
       # if self.df.at[index, 'LK'] is None:   
        self.df.at[index, 'LK'] = lk
            
    def get_dataframe(self):
        return self.df

In [3]:
class MyScanner:
    def __init__(self, timeout, start_serial, end_serial):
        self._scanner = BleakScanner(detection_callback=self.detection_callback)
        self.scanning = asyncio.Event()
        self.timeout = timeout
        self.dc = DataCollector(start_serial, end_serial)
        self.queue_devices_to_connect = Queue()
    def detection_callback(self, device, advertisement_data):
        try:
            if device.name in self.dc.device_names: 
                self.queue_devices_to_connect.put(device)
                self.dc.device_names.remove(device.name)
                print(f'found device with name: {device.name} {self.dc.start_len-len(self.dc.device_names)}/{self.dc.start_len}')
                self.dc.update_MAC(device.name, device.address)
                self.dc.update_RSSI(device.name, advertisement_data.rssi)
                oil_level_raw, battery_voltage, TD_temp_raw = adv_decrypt(advertisement_data.manufacturer_data[3862])
                self.dc.update_oil_level(device.name, oil_level_raw)
                self.dc.update_battery_voltage(device.name, battery_voltage)
                self.dc.update_temp(device.name, TD_temp_raw)
        except Exception as e:
            print(f"Error in callback (scanner): {e}")
    async def run(self):
        self._scanner = BleakScanner(detection_callback=self.detection_callback)
        print(f'\t\tStarted scanning with {self.timeout} seconds timeout...')
        await self._scanner.start()
        self.scanning.set()
        end_time = loop.time() + self.timeout
        while self.scanning.is_set():
            if loop.time() > end_time:
                self.scanning.clear()
                await self._scanner.stop()
                print(f"\t\tScan has timed out so we terminate.")
                if len(self.dc.device_names) == 0:
                    print('All devices have been found.')
                else:
                    print(f"Can't founded devices ({len(self.dc.device_names)*100/self.dc.start_len:2f}%) : {self.dc.device_names}")
                    while True:
                        retry = input("Would you like to retry search? (y/n) ")
                        if retry.lower() == "y":
                            change_timeout = input('Would you like change timeout? (y/n) ')
                            if change_timeout.lower() == "y":
                                self.timeout = int(input('Timeout seconds (int): '))
                                await self.run()
                                break
                            else:
                                await self.run()
                                break
                        elif retry.lower() == "n":
                            return
                        else:
                            print("Invalid input, please enter 'y' or 'n'.")
                await asyncio.sleep(0.01)
            else:
                continue
    def get_dataframe(self):
        return self.dc.get_dataframe()
    def queue_connection(self):
        while not self.queue_devices_to_connect.empty():
            print('Devices in queue:', self.queue_devices_to_connect.queue)
            device = self.queue_devices_to_connect.get()
            assistant = BleakClientAssistant(device)
            hk, lk = assistant.run(b"GA\r")
            self.dc.update_HK(device.name, hk)
            self.dc.update_LK(device.name, lk)

In [4]:
%%time
my_scanner = MyScanner(timeout=10, start_serial=383448, end_serial=384000)
loop = asyncio.get_event_loop()
loop.run_until_complete(my_scanner.run())

		Started scanning with 10 seconds timeout...
found device with name: TD_383448 1/553
Error in callback (scanner): name 'adv_decrypt' is not defined
found device with name: TD_383449 2/553
Error in callback (scanner): name 'adv_decrypt' is not defined
		Scan has timed out so we terminate.
Can't founded devices (99.638336%) : ['TD_383450', 'TD_383451', 'TD_383452', 'TD_383453', 'TD_383454', 'TD_383455', 'TD_383456', 'TD_383457', 'TD_383458', 'TD_383459', 'TD_383460', 'TD_383461', 'TD_383462', 'TD_383463', 'TD_383464', 'TD_383465', 'TD_383466', 'TD_383467', 'TD_383468', 'TD_383469', 'TD_383470', 'TD_383471', 'TD_383472', 'TD_383473', 'TD_383474', 'TD_383475', 'TD_383476', 'TD_383477', 'TD_383478', 'TD_383479', 'TD_383480', 'TD_383481', 'TD_383482', 'TD_383483', 'TD_383484', 'TD_383485', 'TD_383486', 'TD_383487', 'TD_383488', 'TD_383489', 'TD_383490', 'TD_383491', 'TD_383492', 'TD_383493', 'TD_383494', 'TD_383495', 'TD_383496', 'TD_383497', 'TD_383498', 'TD_383499', 'TD_383500', 'TD_38350

Would you like to retry search? (y/n) n
CPU times: total: 21.8 s
Wall time: 44 s


In [5]:
print(1)

1
